
## Introduction

The aim of this assignment is to give you the experience of using real public data and create your own report. The raw data are available in different formats and structures and often contain undesired formatting errors. Statistical data are continuously updated and may even provide API's to stream data instead of taking snapshots. We will be using snapshots in this assignment but to streamline the analysis around possible future snapshots you'll be implementing a 'data-specific' Python class with appropriate methods for data manipulation and visualisation.

**Note:** please direct your questions on this assignment to **r.monajemi@lumc.nl**

## Data: Emissions to air by the Dutch economy

In this assignment we will be using *Emissions to air by the Dutch economy* [CBS StatLine](https://opendata.cbs.nl/portal.html?_la=en&_catalog=CBS&tableId=83300ENG&_theme=1129) dataset. There you'll find the latest release of the dataset, however and older version will be provided via brightspace. Note that the results that shown below are produced using the old datasets: the metadata (`83300ENG_metadata.csv`) and the raw data (`83300ENG_UntypedDataSet_29032023_184145.csv`). 




## Preparation

In [1]:
# find the beginning and end of the dataset
import csv


meta_lines = []
with open("data/83300ENG_metadata.csv", 'r') as f:
    csv_data = csv.reader(f, delimiter=';')
    for row in csv_data:
        meta_lines.append(row)
# print(len(meta_lines))
# print(meta_lines[9])
# for i in range(len(meta_lines)):
#     if "PM10_14" in meta_lines[i]:
#         print(i)

meta_emission = meta_lines[9:26]
column_index = [4, 5, 6, -3]
new_list = [[row[i] for i in column_index] for row in meta_emission]
print(new_list[16])

['PM10_14', 'PM10', 'Particulate matter (PM10 = particulates with diameter less than 10 micrometres). Among other causes, PM10 is formed during the combustion of diesel fuel, other fuels, various industrial processes, and wear processes like the wear of tyres, brake linings, road surface, and railway overhead contact lines. \nPM10 is detrimental to health, penetrates deeply into the lungs.', 'mln kgs']


## Solutions for assignment C

In [244]:
# final edition
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

class GWP:
    # Question 1: read the data
    def __init__(self, path, meta_path):            # g provides the weight of the "unit", kCal the energy of the "unit"
        self._path = path
        self._meta_path = meta_path
        # read the emmission sources
        meta_lines = []
        with open(meta_path, 'r') as f:
            csv_data = csv.reader(f, delimiter=';')
            for row in csv_data:
                meta_lines.append(row)
        header = meta_lines[27]
        header[0]= "DutchEconomy"
        meta_source = meta_lines[28:87]
        self.emission_sources = pd.DataFrame(meta_source,columns = header)
        # read the emmission
        header_emission = ["", "Title",	"Description", "Unit"]
        meta_emission = meta_lines[9:26]
        column_index = [4, 5, 6, -3]
        new_list = [[row[i] for i in column_index] for row in meta_emission]
        self.emission = pd.DataFrame(new_list,columns = header_emission)
        # read the data
        self.data = pd.read_csv(path,sep=";")
        # get the date
        self.data["Date"] = pd.to_datetime(self.data["Periods"].str[0:4])
        # merge the data with the source
        self.data = self.data.merge(self.emission_sources, how='left', on='DutchEconomy')
        self.data.drop('Description', inplace=True, axis=1)
        self.data.rename(columns={'Title': 'Source'}, inplace=True)
    # Question 2: adjust the data
        # Convert N2O_4 and CH4_5 to CO2 equivalents.
        self.data["N2O_4"] *= 298 
        self.data["CH4_5"] *= 25
        # Add a new variable HFC
        self.data["HFC"] =  self.data["GreenhouseGasEquivalents_6"] - self.data["TotalCO2_1"] - self.data["N2O_4"] - self.data["CH4_5"]
    # Question 3: define plot
    def plot(self,emission,source):
        data_draw = pd.DataFrame(columns=self.data.columns)
        for s in source:
            for j in range(len(self.data.index)):
                if s in self.data.iloc[j]["Source"].lower() or s in self.data.iloc[j]["DutchEconomy"].lower():
                    data_draw.loc[len(data_draw)] = self.data.iloc[j]

        emission_df = data_draw[emission]
        id_vars = ['Date', 'Source']
        select_id_vars = data_draw[id_vars]
        merged_df = pd.concat([select_id_vars, emission_df], axis=1)
        melt_data = pd.melt(frame=merged_df, id_vars=id_vars, var_name='Emission', value_name='Value')
        sns.lineplot(data=melt_data, x='Date', y='Value', hue='Source', style='Emission')
        plt.legend(bbox_to_anchor=(1.05, 0.5), loc='center left')
    # Question 4: define get_emission
    def get_emission(self, category):
        emission_category = {"ghg": ["TotalCO2_1", "CO2ExclBiomass_2", "CO2Biomass_3", "N2O_4", "CH4_5", "GreenhouseGasEquivalents_6", "HFC"], 
                        "acid": ["NOx_7", "SO2_8", "NH3_9", "AcidificationEquivalents_10"], 
                        "ozone": ["CFK12Equivalents_11"], 
                        # HFC moved from air to ghg
                        "air": ["CO_12", "NMVOC_13", "PM10_14"]}
        emissions = emission_category[category]
        return emissions        
    # Question 5: define summary
    def summary(self, date=None):
        if date:

            # If date is given, print descriptive statistics for emissions
            datetime_l = np.datetime64(f'{date}-01-01')
            # datetime_u = np.datetime64(f'{max}-01-01')
            data_date = self.data[self.data["Date"] >= datetime_l]

            column_names_list = gwp.data.columns.tolist()
            emmisions_names = column_names_list[3:-3]
            min_val = [0 for _ in range(len(emmisions_names))]
            mean_val = [0 for _ in range(len(emmisions_names))]
            max_val = [0 for _ in range(len(emmisions_names))]
            q1 = [0 for _ in range(len(emmisions_names))]
            q2 = [0 for _ in range(len(emmisions_names))]
            q3 = [0 for _ in range(len(emmisions_names))]

            for i in range(len(emmisions_names)):
                
                # Extract relevant statistics
                min_val[i] = data_date[emmisions_names[i]].min()
                q1[i] = np.percentile(data_date[emmisions_names[i]], 25)
                q2[i] = data_date[emmisions_names[i]].median()
                mean_val[i] = data_date[emmisions_names[i]].mean()
                q3[i] = np.percentile(data_date[emmisions_names[i]], 75)
                max_val[i] = data_date[emmisions_names[i]].max()

                # Create a new DataFrame for formatted output
            df_stats = {
                'min': min_val,
                '1st quartile': q1,
                '2nd quartile': q2,
                'mean': mean_val,
                '3rd quartile': q3,
                'max': max_val
            }

            desc_stats =pd.DataFrame(df_stats, index=emmisions_names)

            print(desc_stats)
        else:
            # Print general information about the data
            print("Data Path:", self._path)
            print("Metadata Path:", self._meta_path)
            print("Shape of Data:", self.data.shape)
            print("Number of Emission Sources:", self.data["Source"].nunique())
            print("Number of Emission Variables:", len(self.data["Source"]))
            print("First Date:", self.data["Date"].min())
            print("Latest Date:", self.data["Date"].max())


In [245]:
gwp = GWP(path="data/83300ENG_UntypedDataSet_29032023_184145.csv", meta_path='data/83300ENG_metadata.csv')

## Assignment

#### 1) Class GWP [4p].

- Create the class `GWP` (Global Warming Potential) which is initialised with raw data along with the meta information.
- Create class attributes [emissions](emissions.md) and [emission_sources](emission_sources.md) from the metadata.
- Additional attributes:
    - `Date` :  of type timestamp created from the variable `Periods`
    - `Source` : holding the `Title` values of the emission source in the metadata.

In [205]:
gwp = GWP(path="data/83300ENG_UntypedDataSet_29032023_184145.csv", meta_path='data/83300ENG_metadata.csv')

In [ ]:
gwp.data.head()

In [ ]:
gwp.emission.head()

In [ ]:
gwp.emission_sources.head()

In [ ]:
gwp.data.dtypes

#### 2) Greenhouse equivalents [1p].

The [greenhouse equivalents](https://www.cbs.nl/en-gb/news/2020/19/uitstoot-broeikasgassen-3-procent-lager-in-2019/co2-equivalents) is a measure to relate othe other gases to CO2.

- Convert `N2O_4` and `CH4_5` in `GWP.data` to CO2 equivalents.
- Add a new variable `HFC` to the GWP.data which contains the fluorine (chlorine) gases CO2 equivalents. HFC stands for hydrofluorocarbons (see [Fluorinated gases](https://en.wikipedia.org/wiki/Fluorinated_gases)). The emission has been added to the total greenhouse gas equivalent but not specified as separate variable. See also *greenhouse gas equivalents* description in the metadata.

#### 3) Implement a plot method that produces the figure below. [3p]

**Synopsis:** &nbsp; &nbsp;**<tt>GWP.plot(emission, source)</tt>**

    - emission: is a list of emissions to be drawn
    - source: list of Dutch economy sector id's or a pattern

In [ ]:
gwp.plot(emission=['CO_12', 'NMVOC_13'],source=['education', 'government','320000' ]);


#### 4) Emission categories [1p]

There are 4 categories of emissions:
   - Greenhouse gases (climate change) {TotalCO2_1, CO2ExclBiomass_2, CO2Biomass_3, N2O_4, CH4_5, GreenhouseGasEquivalents_6}
   - Acidification {NOx_7, SO2_8, NH3_9, AcidificationEquivalents_10}
   - Ozone layer depletion {CFK12Equivalents_11}
   - Other air pollution {CO_12, NMVOC_13, PM10_14, HFC}

Implement the method `get_emission` with the following specification:

**Synopsis:** &nbsp; &nbsp;**<tt>GWP.get_emission(emission_category)</tt>**

   - emission_category : {ghg, acid, ozone, air}
   - return value : list of variable names of the given category.


In [ ]:
gwp.plot(emission=gwp.get_emission('air'),source=['transport']);

#### 5) Summary [1p]

Implement the `summary` method which gives `general information` about the data such as:
- data and meta file paths
- shape of the data
- number of emission sources.
- number of emission variables
- first and latest date


**Synopsis:** &nbsp; &nbsp;**<tt>GWP.summary(date)</tt>**
   - `date`     : tuple of dates (min,[max]).

If `date` is given then print a DataFrame of descriptive statistics with columns {min, 1st quartile, 2nd quartile, mean 3rd quartile, max} and emission as row indices. In this case do not output the `general information`. Note, for the `date` method argument, in case 'max' is omitted then the summary is given for a single year.


In [231]:
gwp.summary()

Data Path: data/83300ENG_UntypedDataSet_29032023_184145.csv
Metadata Path: data/83300ENG_metadata.csv
Shape of Data: (1888, 20)
Number of Emission Sources: 59
Number of Emission Variables: 1888
First Date: 1990-01-01 00:00:00
Latest Date: 2021-01-01 00:00:00


In [246]:
gwp.summary(1990)

                             min  1st quartile  2nd quartile          mean  \
TotalCO2_1                   0.0       430.750       1770.50  14231.980932   
CO2ExclBiomass_2             0.0       381.000       1630.50  13531.443856   
CO2Biomass_3                 0.0         0.600          5.90    700.532415   
N2O_4                        0.0         0.000          0.00    889.075424   
CH4_5                        0.0         2.500          7.50   1074.267744   
GreenhouseGasEquivalents_6   0.0       495.000       2306.00  16647.032839   
NOx_7                        0.0         0.700          4.40     43.100794   
SO2_8                        0.0         0.000          0.20     10.776695   
NH3_9                        0.0         0.000          0.00      8.949417   
AcidificationEquivalents_10  0.0         0.000          0.10      1.793114   
CFK12Equivalents_11          0.0         0.000          0.00     61.959905   
CO_12                        0.0         1.175          4.00    

#### 6) Bonus point

Add a new plot method to the class which gives a different kind of information than the one given in GWP.plot.